<a href="https://colab.research.google.com/github/ezemaudet/TP_Analisis_Sentimiento/blob/main/transcripcion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Transcripcion**

In [ ]:
!pip install python-dotenv
!pip install openai-whisper
from dotenv import load_dotenv
import os
import requests
import pandas as pd
import whisper


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 800.5/800.5 kB 30.5 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 89.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 77.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 49.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 6.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 68.0 MB/s

In [ ]:
# Ruta al archivo .env dentro de tu Google Drive
dotenv_path = "/.env.development"

load_dotenv(dotenv_path)

# Configuración de la API
API_KEY = os.getenv("API_KEY_vozelia")
TENANT = os.getenv("TENANT_vozelia")
START_DATE = "2025-05-16"
END_DATE = "2025-05-17"
TIME=60
prueba=3

# URL para obtener los registros de llamadas en CSV
url = f"https://pbxone.vozelia.com.pa/pbx/proxyapi.php?key={API_KEY}&reqtype=INFO&info=CDRS&format=csv&tenant={TENANT}&start={START_DATE}&end={END_DATE}"

response = requests.get(url)

if response.status_code == 200:
    with open("llamadas.csv", "wb") as f:
        f.write(response.content)

    print("CDR descargado exitosamente.")

    # Cargar CSV en pandas para visualizar datos
    df = pd.read_csv("llamadas.csv")
    print(df.head())  # Mostrar las primeras filas

else:
    print("Error al obtener las llamadas:", response.text)

CDR descargado exitosamente.
   accountcode         ID                start               answer  \
0        11279  229719256  2025-05-16 07:21:29  0000-00-00 00:00:00   
1        11279  229719257  2025-05-16 07:21:32  2025-05-16 07:22:04   
2        11279  229719346  2025-05-16 07:25:14  0000-00-00 00:00:00   
3        11279  229719712  2025-05-16 07:32:35  0000-00-00 00:00:00   
4        11279  229719713  2025-05-16 07:32:38  0000-00-00 00:00:00   

                   end          clid    realsrc  firstdst  duration  billsec  \
0  2025-05-16 07:21:32  "" <2336082>  102-11279  67359102         2        0   
1  2025-05-16 07:22:05  "" <2336082>  102-11279  67359102        32        0   
2  2025-05-16 07:25:49  "" <2336082>  102-11279  67359102        35        0   
3  2025-05-16 07:32:38  "" <2336082>  102-11279  67359102         3        0   
4  2025-05-16 07:33:09  "" <2336082>  102-11279  67359102        30        0   

   ...       dst  lastapp                   srcCallID  \
0  ...

In [ ]:
# 1. Convertir a datetime y calcular duración
df['answer'] = pd.to_datetime(df['answer'], errors='coerce')
df['end'] = pd.to_datetime(df['end'], errors='coerce')
df['duracion_timedelta'] = df['end'] - df['answer']
df['duracion_segundos'] = df['duracion_timedelta'].dt.total_seconds()

# 2. Filtrar llamadas con duración > XX segundos
df_filtrado = df[df['duracion_segundos'] > TIME].head(prueba).copy()
df_filtrado['transcripcion'] = None  # Crear columna vacía para la transcripción



<ipython-input-9-3f119ad3443a>:2: UserWarning: Could not infer format, so each element will be parsed individually, falling back to `dateutil`. To ensure parsing is consistent and as-expected, please specify a format.
  df['answer'] = pd.to_datetime(df['answer'], errors='coerce')


In [ ]:
#model = whisper.load_model("medium")
model = whisper.load_model("base")

100%|███████████████████████████████████████| 139M/139M [00:01<00:00, 85.0MiB/s]


In [ ]:
df_filtrado

,accountcode,ID,start,answer,end,clid,realsrc,firstdst,duration,billsec,...,linkedid,peeraccount,originateid,cc_country,cc_network,pincode,cc_buy,duracion_timedelta,duracion_segundos,transcripcion
5,11279,229719767,2025-05-16 07:33:31,2025-05-16 07:33:34,2025-05-16 07:34:46,"""DANIELA RODRIGUEZ"" <102>",102-11279,301,74,71,...,cpbx06.vozelia.com.pa-1747398811.12182967,11279.0,NaN,NaN,NaN,NaN,0.0,0 days 00:01:12,72.0,None
9,11279,229721670,2025-05-16 08:03:32,2025-05-16 08:03:36,2025-05-16 08:05:57,"""YEXICA"" <101>",101-11279,100,144,141,...,cpbx06.vozelia.com.pa-1747400612.12188995,11279.0,NaN,NaN,NaN,NaN,0.0,0 days 00:02:21,141.0,None
10,11279,229721855,2025-05-16 08:06:25,2025-05-16 08:06:29,2025-05-16 08:07:52,"""Brigida Campbell"" <100>",100-11279,101,87,82,...,cpbx08.vozelia.com.pa-1747400785.20933996,11279.0,NaN,NaN,NaN,NaN,0.0,0 days 00:01:23,83.0,None


In [ ]:
# 3. Iterar sobre cada fila y procesar grabación
for index, row in df_filtrado.iterrows():
    uniqueid = row['uniqueid']
    RECORDING_ID = uniqueid

    print(f"Descargando grabación para uniqueid {uniqueid}...")

    url = f"https://pbxone.vozelia.com.pa/pbx/proxyapi.php?key={API_KEY}&reqtype=INFO&info=recording&id={RECORDING_ID}&tenant={TENANT}"

    response = requests.get(url)

    if response.status_code == 200:
        # Guardar temporalmente la grabación
        with open("grabacion.wav", "wb") as f:
            f.write(response.content)

        print("✓ Grabación descargada. Iniciando transcripción...")

        try:
            result = model.transcribe("grabacion.wav", language="es")
            transcripcion = result["text"]
            df_filtrado.at[index, 'transcripcion'] = transcripcion
            print(f"✓ Transcripción completada para {uniqueid}.")
        except Exception as e:
            print(f"⚠️ Error al transcribir {uniqueid}: {e}")

        # Borrar el archivo de audio para ahorrar memoria
        os.remove("grabacion.wav")

    else:
        print(f"❌ Error al obtener grabación de {uniqueid}: {response.text}")



Descargando grabación para uniqueid cpbx06.vozelia.com.pa-1747398811.12182967...
✓ Grabación descargada. Iniciando transcripción...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✓ Transcripción completada para cpbx06.vozelia.com.pa-1747398811.12182967.
Descargando grabación para uniqueid cpbx06.vozelia.com.pa-1747400612.12188995...
✓ Grabación descargada. Iniciando transcripción...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✓ Transcripción completada para cpbx06.vozelia.com.pa-1747400612.12188995.
Descargando grabación para uniqueid cpbx08.vozelia.com.pa-1747400785.20933996...
✓ Grabación descargada. Iniciando transcripción...


/usr/local/lib/python3.11/dist-packages/whisper/transcribe.py:126: UserWarning: FP16 is not supported on CPU; using FP32 instead
  warnings.warn("FP16 is not supported on CPU; using FP32 instead")


✓ Transcripción completada para cpbx08.vozelia.com.pa-1747400785.20933996.


In [ ]:
# Mostrar un resumen
print("\n📋 Muestras con transcripciones:")
print(df_filtrado[['ID', 'uniqueid', 'duracion_segundos', 'transcripcion']].head())

# (Opcional) Guardar en CSV o sobrescribir `df`
df.update(df_filtrado[['transcripcion']])
df_filtrado.to_csv("/transcripciones.csv", index=False)




📋 Muestras con transcripciones:
           ID                                   uniqueid  duracion_segundos  \
5   229719767  cpbx06.vozelia.com.pa-1747398811.12182967               72.0   
9   229721670  cpbx06.vozelia.com.pa-1747400612.12188995              141.0   
10  229721855  cpbx08.vozelia.com.pa-1747400785.20933996               83.0   

                                        transcripcion  
5    ya voy bajando, ¿qué pasó? El hielo que está ...  
9    según las partisanas yo meimo te voy a squeez...  
10   ¿Qué no? Hay una misma oportunidad que va a r...  
